In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib.pyplot import figure
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import statistics
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
import os

# Import data

In [ ]:

trainData= pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
testData = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
out1=testData['Id']

# Exploratory Data Analysis (EDA)

In [ ]:
trainData.head()

In [ ]:
testData.head()

In [ ]:
trainData.columns

In [ ]:
# checking for null values for both training and testing data
trainData.isnull().sum().sum()

In [ ]:
testData.isnull().sum().sum()

In [ ]:
# Checking for class imbalance
trainData.groupby("Cover_Type").Cover_Type.hist()
trainData['Cover_Type'].value_counts()

In [ ]:
## cover type = 5 has just 1 dataset for training. This is of very little help to us. So, dropping it.
trainData.drop(trainData[trainData['Cover_Type']==5].index,inplace=True)

In [ ]:
# Dividing the training dataset into continuous and one-hot encoded ones for futher EDA
trainData.columns
trainDataSet1 = trainData[['Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points','Cover_Type']]

trainDataSet2=trainData[['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40',
       'Cover_Type']]

In [ ]:
# Doing a box plot for first level visualisation of continuous features
for column in trainDataSet1:
    plt.figure()
    sns.boxplot(y = trainDataSet1[column],
            x = trainDataSet1['Cover_Type'])

In [ ]:
# Checking the correlation matrix
corr_matrix1 = trainDataSet1.corr()
corr_matrix2 = trainDataSet2.corr()
corr_matrix3 = trainData.corr()

In [ ]:
# plotting correlation heatmap
dataplot1 = sns.heatmap(corr_matrix1, cmap="YlGnBu", annot=False)

In [ ]:
# plotting correlation heatmap
dataplot2 = sns.heatmap(corr_matrix2, cmap="YlGnBu", annot=False)

In [ ]:
dataplot3 = sns.heatmap(corr_matrix3, cmap="YlGnBu", annot=False)

In [ ]:
## dropping columns 'Soil_Type7' and 'Soil_Type15' and 'Id' from training dataset

trainData=trainData.drop(['Id','Soil_Type7','Soil_Type15'],axis=1)
testData=testData.drop(['Id','Soil_Type7','Soil_Type15'],axis=1)

# Pre-processing

In [ ]:
# Prepping the data 
# separating features and their labels
X=trainData.drop('Cover_Type',axis=1)
y=trainData['Cover_Type']


In [ ]:
# Standardisation
# for training data
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)

In [ ]:
# for testing Data
scaler.transform(testData)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_memory = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_memory = df.memory_usage().sum() / 1024**2
    if verbose: 
        print(f"Memory usage of dataframe after reduction {end_memory} MB")
        print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
X = reduce_mem_usage(X)
testData = reduce_mem_usage(testData)

In [ ]:
tps_clf = RandomForestClassifier(random_state=42)

In [ ]:
tps_clf.fit(X,y)

In [ ]:
del trainData
del trainDataSet1
del trainDataSet2
del corr_matrix1
del corr_matrix2
del corr_matrix3

In [ ]:
y_pred=tps_clf.predict(testData)

In [ ]:
out1 = pd.DataFrame(out1, columns = ['Id'])
y_pred = pd.DataFrame(y_pred, columns = ['Cover_Type'])
frame3 = pd.concat([out1, y_pred], axis=1)
frame3.to_csv('kaggle_TPS_submission.csv')